In [ ]:
import datetime
import pandas as pd
import datetime as dt

In [ ]:
# TitreGenre : contient les informations sur les films (titre, genre, année de sortie, etc.)
# DescriptionEquipeFilm : contient les informations sur les membres de l'équipe de chaque film (IDfilm, IDpersonne, réalisateur, acteur, actrice)
# NomActeur : contient les informations sur les acteurs et actrices (nom et IDpersonne)
# TitreRegion : contient les informations sur les titres des films en fonction de la région (IDfilm, titre, région)

In [ ]:
# Description des merges pour la création de la base de données finale
# TitreGenre merge avec ?????
#

In [ ]:
# Fichier principal où les filtres sont appliqués qui servira de base pour les merges avec les autres fichiers
# lecture du fichier title.basics.tsv.gz depuis le lien fourni et filtrage des données pour ne conserver que les films qui nous intéressent

TitreGenre = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t",low_memory=False)
TitreGenre = TitreGenre[TitreGenre['titleType'].isin(["movie"])]
TitreGenre['runtimeMinutes'] = pd.to_numeric(TitreGenre['runtimeMinutes'],errors='coerce')
TitreGenre = TitreGenre[(TitreGenre["runtimeMinutes"] > 40)]
TitreGenre["startYear"] = pd.to_datetime(TitreGenre["startYear"], errors="coerce", format="%Y")
TitreGenre = TitreGenre[TitreGenre["startYear"] > pd.Timestamp("1990-01-01")]
TitreGenre["genres_list"] = TitreGenre["genres"].str.split(",")
TitreGenre = TitreGenre.drop("genres",axis=1)
TitreGenre = TitreGenre.drop("endYear",axis=1)
TitreGenre = TitreGenre[TitreGenre["isAdult"] == 0]
genres_exclus = {
    "Horror",
    "Music",
    "Sci-Fi",
    "Sport",
    "Musical",
    "News",
    "Adult",
    "Western",
    "Reality-TV",
    "Talk-Show",
    "Game-Show",
}

TitreGenre = TitreGenre[
    TitreGenre["genres_list"].apply(
        lambda genres:any(g not in genres_exclus for g in genres))
]

In [ ]:
# Lecture du fichier title.principals.tsv.gz et sélection des colonnes pertinentes pour merge
NomActeur = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz",
                        na_values="\\N", sep="\t", low_memory=False, usecols=["nconst", "primaryName"])

In [ ]:
# Lecture du fichier title.principals.tsv.gz et sélection des colonnes pertinentes pour merge 
# et filtrage pour ne conserver que les réalisateurs, acteurs et actrices
DescriptionEquipeFilm = pd.read_csv(
        "https://datasets.imdbws.com/title.principals.tsv.gz",
        sep="\t",
        na_values="\\N",
        usecols=["tconst", "ordering", "nconst", "category"],
        low_memory=False
    )
DescriptionEquipeFilm = DescriptionEquipeFilm.loc[DescriptionEquipeFilm["category"].isin(["director", "actor", "actress"])]

In [ ]:
# Lecture du fichier title.akas.tsv.gz et sélection des colonnes pertinentes et filtrage pour ne conserver que les titres avec une région FR
TitreRegion = pd.read_csv("https://datasets.imdbws.com/title.akas.tsv.gz",
                           sep="\t", na_values="\\N",low_memory=False, usecols=["titleId","title","region"])
TitreRegion = TitreRegion[TitreRegion["region"] == "FR"]